In [0]:
spark

In [0]:
dbutils.secrets.help()

Provides utilities for leveraging secrets within notebooks.
Databricks documentation for more info.
 get(scope: String, key: String): String -> Gets the string representation of a secret value with scope and key getBytes(scope: String, key: String): byte[] -> Gets the bytes representation of a secret value with scope and key list(scope: String): Seq -> Lists secret metadata for secrets within a scope listScopes: Seq -> Lists secret scopes

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='databricks-scope')]

In [0]:
dbutils.secrets.list(scope = "databricks-scope")

[SecretMetadata(key='zecdatastorage-client-id'),
 SecretMetadata(key='zecdatastorage-secret-key'),
 SecretMetadata(key='zecdatastorage-tenant-id')]

In [0]:
dbutils.secrets.get(scope="databricks-scope",key="zecdatastorage-client-id")

'[REDACTED]'

In [0]:
# Connect to Azure Data Lake Storage Gen2 using service principle - key valut , scope
# https://learn.microsoft.com/en-us/azure/databricks/connect/storage/tutorial-azure-storage

clinet_id = dbutils.secrets.get(scope="databricks-scope",key="zecdatastorage-client-id")
tenant_id = dbutils.secrets.get(scope="databricks-scope",key="zecdatastorage-tenant-id")
secreat_id =  dbutils.secrets.get(scope="databricks-scope",key="zecdatastorage-secret-key")

In [0]:
# spark.conf.set("fs.azure.account.auth.type.zecdatastorage.dfs.core.windows.net", "OAuth")
# spark.conf.set("fs.azure.account.oauth.provider.type.zecdatastorage.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
# spark.conf.set("fs.azure.account.oauth2.client.id.zecdatastorage.dfs.core.windows.net", clinet_id)
# spark.conf.set("fs.azure.account.oauth2.client.secret.zecdatastorage.dfs.core.windows.net", secreat_id)
# spark.conf.set("fs.azure.account.oauth2.client.endpoint.zecdatastorage.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": clinet_id,
          "fs.azure.account.oauth2.client.secret": secreat_id,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://demo@zecdatastorage.dfs.core.windows.net/",
  mount_point = "/mnt/zecdatastorage/demo",
  extra_configs = configs)

True

In [0]:
display(dbutils.fs.ls("abfss://demo@zecdatastorage.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://demo@zecdatastorage.dfs.core.windows.net/employees.csv,employees.csv,3777,1732969045000
abfss://demo@zecdatastorage.dfs.core.windows.net/file5.json,file5.json,870972,1733122713000


In [0]:
# display(dbutils.fs.ls("abfss://demo@zecdatastorage.dfs.core.windows.net"))
display(dbutils.fs.ls("/mnt/zecdatastorage/demo"))

path,name,size,modificationTime
dbfs:/mnt/zecdatastorage/demo/employees.csv,employees.csv,3777,1732969045000
dbfs:/mnt/zecdatastorage/demo/file5.json,file5.json,870972,1733122713000


In [0]:
# display(spark.read.csv("abfss://demo@zecdatastorage.dfs.core.windows.net/employees.csv"))
display(spark.read.csv("/mnt/zecdatastorage/demo/employees.csv"))

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10
EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
198,Donald,OConnell,DOCONNEL,650.507.9833,21-JUN-07,SH_CLERK,2600,-,124,50
199,Douglas,Grant,DGRANT,650.507.9844,13-JAN-08,SH_CLERK,2600,-,124,50
200,Jennifer,Whalen,JWHALEN,515.123.4444,17-SEP-03,AD_ASST,4400,-,101,10
201,Michael,Hartstein,MHARTSTE,515.123.5555,17-FEB-04,MK_MAN,13000,-,100,20
202,Pat,Fay,PFAY,603.123.6666,17-AUG-05,MK_REP,6000,-,201,20
203,Susan,Mavris,SMAVRIS,515.123.7777,07-JUN-02,HR_REP,6500,-,101,40
204,Hermann,Baer,HBAER,515.123.8888,07-JUN-02,PR_REP,10000,-,101,70
205,Shelley,Higgins,SHIGGINS,515.123.8080,07-JUN-02,AC_MGR,12008,-,101,110
206,William,Gietz,WGIETZ,515.123.8181,07-JUN-02,AC_ACCOUNT,8300,-,205,110


In [0]:
# df = spark.read.format("csv").option("inferSchema",True).option("header",True).load("abfss://demo@zecdatastorage.dfs.core.windows.net/employees.csv")

df = spark.read.format("csv").option("inferSchema",True).option("header",True).load("/mnt/zecdatastorage/demo/employees.csv")

In [0]:
df.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [0]:
df.count()

50

In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/mnt/zecdatastorage/demo,abfss://demo@zecdatastorage.dfs.core.windows.net/,
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,
/volume,DbfsReserved,


In [0]:
# dbutils.fs.unmount("/mnt/zecdatastorage/demo")

In [0]:
print("Succsess")

Succsess
